In [ ]:
import warnings
warnings.filterwarnings("ignore")
from bs4 import BeautifulSoup
import urllib.request as req
import requests
from tqdm import tqdm_notebook
import pandas as pd
import numpy as np
import urllib.parse as par

In [ ]:
#TODO : 도쿄 치안 데이터 : 치안 관련 키워드로 일본 도쿄 공공 데이터 수집
#* 치안('治安'), 범죄('犯罪'), 경찰('警察'), 범죄 예방('犯罪予防'), CCTV('監視'), 범죄자('犯罪者'), 안전('安全'), 교통 안전('交通安全'), 경비원('警備員'), 범죄 통계('犯罪統計')
#* 조회해보니 치안과 관련된 데이터 중 유용한 데이터는 '범죄('犯罪')' 키워드 뿐, 다른 데이터는 치안과 무관하거나 '범죄'키워드와 중복된다. 
#* 범죄('犯罪') 관련 키워드만 크롤링
keyword = '犯罪'

In [ ]:
def encoded_text(keyword):
    encoded = par.quote(keyword)
    return encoded

In [ ]:
import requests
from bs4 import BeautifulSoup

def get_page_info(keyword):
    #* 첫 페이지 요청
    encoded_keyword = encoded_text(keyword)
    url = f'https://catalog.data.metro.tokyo.lg.jp/dataset?q={encoded_keyword}&sort=score+desc%2C+metadata_modified+desc'
    response = requests.get(url)
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')

    #* 해당 키워드 전체 페이지 수
    totalPageNum = len(soup.select('ul.pagination > li')) - 1
    #print(f"{keyword}의 전체 페이지 수:", totalPageNum)
    return totalPageNum

In [ ]:
def total_connect_url(keyword):
    wholeurl = []
    for idx in range(1, get_page_info(keyword) + 1):
        encoded_keyword = encoded_text(keyword)
        url = f'https://catalog.data.metro.tokyo.lg.jp/dataset?q={encoded_keyword}&sort=score+desc%2C+metadata_modified+desc&page={idx}'
        wholeurl.append(url)
    return wholeurl

In [ ]:
datanumList = []
def get_connect_url(keyword):
    for url in total_connect_url(keyword):
        code = requests.get(url)
        soup = BeautifulSoup(code.text, 'html.parser')
        datanum = soup.select('ul.dataset-list.list-unstyled h3.dataset-heading > a')
        connectBaseurl = 'https://catalog.data.metro.tokyo.lg.jp'
        for idx in range(len(datanum)):
            datanumList.append(connectBaseurl + datanum[idx].attrs['href'])
    #print(len(datanumList))
    return datanumList

In [ ]:
# import chardet

# with open('tokyo/fuc22a12pol01police.csv', 'rb') as rawdata:
#     result = chardet.detect(rawdata.read(10000))

# # check what the character encoding might be
# print(result)

In [ ]:
import requests.adapters

#* 어댑터 사용 설정
adapter = requests.adapters.HTTPAdapter()
session = requests.Session()
session.mount("http://", adapter)
session.mount("https://", adapter)
get_csv_url = []

try:
    url = get_connect_url(keyword)
    for idx in tqdm_notebook(range(len(url))):
        response = session.get(url[idx])
        soup = BeautifulSoup(response.text, 'lxml')
        tmp = soup.select('ul.resource-list > li > a')
        baseurl = 'https://catalog.data.metro.tokyo.lg.jp'
        for link in tmp:
            get_csv_url.append(baseurl + link.get('href'))
        
except requests.exceptions.RequestException as e:
    print("오류 발생:", e)

In [ ]:
import requests
from tqdm import tqdm_notebook
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
#* urllib3 라이브러리에서 발생하는 InsecureRequestWarning 경고 메시지를 비활성화하는 역할
#* 일반적으로 웹 요청 시 SSL 인증서의 유효성을 검사하고 경고 메시지를 표시하는데, 
#* InsecureRequestWarning은 SSL 인증서의 유효성 검사에 실패하여 보안상의 위험이 발생할 수 있음을 경고하는 메시지
#* disable_warnings() 함수는 urllib3 라이브러리에서 경고 메시지를 비활성화하는 데 사용
#* 이를 사용하여 InsecureRequestWarning 경고 메시지를 무시하고 프로그램을 실행가능
#* 하지만 주의해야 할 점은 이렇게 경고 메시지를 비활성화하면 보안 위험을 감수해야 함
#*  verify=False를 사용하여 인증서 검증을 비활성화

final_csv_url = []

for idx in tqdm_notebook(range(len(get_csv_url))):
    url = get_csv_url[idx]
    try:
        code = requests.get(url, timeout=10, verify=False)
        soup = BeautifulSoup(code.text, 'html.parser')
        urlcsv = soup.select_one('p.text-muted.ellipsis > a').attrs['href']
        final_csv_url.append(urlcsv)
    except Exception as e:
        print(f'||{urlcsv}|| 링크가 존재하지 않습니다.')
        print(e)

In [ ]:
import os

download_directory = 'tokyo'

if not os.path.exists(download_directory):
    os.makedirs(download_directory)

In [ ]:
import ssl
import socket
from urllib.error import URLError
import pandas as pd

#* ssl은 SSL 인증서 관련 작업을 위함
#* socket은 연결 시간 초과와 관련된 예외 처리 위함

ssl._create_default_https_context = ssl._create_unverified_context
#*  SSL 인증서 검증을 비활성화, 이는 SSL 인증서가 검증되지 않아도 요청을 계속할 수 있게함
for url in tqdm_notebook(final_csv_url):
    try:
        if url.endswith('.csv'):
            file_name = url.split("/")[-1]
            pd.read_csv(url, encoding='SHIFT_JIS').to_csv('tokyo/' + file_name, encoding='SHIFT_JIS')
    except URLError as e: #* URLError 예외가 발생한 경우, 연결 시간 초과(socket.timeout)인지 확인
        if isinstance(e.reason, socket.timeout): #* isinstance(e.reason, socket.timeout)를 사용하여 예외의 reason이 socket.timeout인지 확인
            print(f"{file_name} 다운로드 중 연결 시간 초과가 발생했습니다.")
        else:
            print(f"{file_name} 다운로드 및 저장 중 오류가 발생했습니다: {e}")
    except Exception as e:
        print(f"{file_name} 다운로드 및 저장 중 오류가 발생했습니다: {e}")